## Imports

In [1]:
import numpy as np
import time
import math
import random
import pandas as pd
from operator import add
import keras

from keras.engine.topology import Input
from keras.engine.training import Model
from keras.layers.convolutional import Conv2D
from keras.layers.core import Activation, Dense, Flatten
from keras.layers.merge import Add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
"""
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, Dense, Flatten
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
"""
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.
W0124 15:40:05.476831  6304 __init__.py:308] Limited tf.compat.v2.summary API due to missing TensorBoard installation.


## Game

In [2]:
class Game:
    def __init__(self,all_cards=None,turn = 1, score1 = 0,score2 = 0):
        self.finish = False
        r=0
        i=0
        if(all_cards == None):
            self.all_cards = [[],[],[],[]]
            nums = list(range(0,52))
            #print(nums)
            random.shuffle(nums)
            for n in nums:
                self.all_cards[math.floor(i/13)].append(n)
                i+=1
        #print(self.all_cards)
        self.deck = []
        self.round = 1
        self.pack = 1
        self.ATO = random.randint(0,3)
        self.finish = False
        self.turn = 1
        self.ATO_count = random.randint(7,10)
        self.score1 = 0
        self.score2 = 0
        self.played_cards = []
    def play_card(self,which):
        #print((self.turn-1))
        #print(which)
        #print(self.all_cards[self.turn-1])
        if(which>=len(self.all_cards[self.turn-1])):which = 0
        v = self.all_cards[self.turn-1][which]
        if(not self.is_playable(v)):
            return -1
        self.deck.append(self.all_cards[self.turn-1][which])
        self.played_cards.append(self.all_cards[self.turn-1][which])
        del[self.all_cards[self.turn-1][which]]
        self.pack+=1
        self.turn+=1
        if(self.turn>4): self.turn=1
        if(self.pack>4): 
            self.end_round()
            self.pack = 1
            self.deck.clear()
            self.round += 1
            if(len(self.all_cards[0])==0):
                self.finish = True
        return 1
    def is_playable(self,which):
        if len(self.deck) == 0:
            return True
        tabclr = math.floor(self.deck[0]/13)
        clr = math.floor(which/13)
        if(tabclr == clr):
            return True
        if(which in self.all_cards[self.turn-1]):
            return True
        return False
    def get_val(self,card):
        clr = math.floor(card/13)
        val = card%13
        if(val==0): val=13
        if(clr==self.ATO): val*= 10
        return val
    def end_round(self):
        besti = -1
        i = 0
        maxi = -9990
        for crd in self.deck:
            if(self.get_val(crd)>maxi):
                maxi = self.get_val(crd)
                besti = i
            i += 1
        self.turn = besti +1
        if(besti==0 or besti == 2):
            self.score1 += 1
        else:
            self.score2 += 1
        

## Player

In [3]:
class Player:
    def __init__(self,PType="human",agent = None,depth=3):
        self.type = PType
        self.depth = depth
        if(PType=="DQN"):
            if(agent != None): self.agent = agent
            else: self.agent = DQNAgent()
    def get_move(self,game,leng):
        if(self.type == "human"):
            return int(input("Enter move:"))
        elif(self.type == "random"):
            #print("LPO")
            #print(leng)
            return np.random.randint(0,leng-1)
        elif(self.type=="DQN"):
            return np.argmax(self.agent.model.predict(np.array(game.board).reshape((1,10)))[0])
        elif(self.type=="minmax"):
            return min_max(game.board,self.depth,game.turn,game.score1,game.score2)
        else:
            return 0

## Run Games

In [4]:
def run(player1,player2,player3,player4,numGames):
    win1 = 0
    win2 = 0
    tie = 0
    for gm in range(numGames):
        print(gm+1)
        g=Game()
        while(not g.finish):
            r=-1
            while(r==-1):
                if(g.turn==1):m =   player1.get_move(g,len(g.all_cards[0]))
                elif(g.turn==2):m = player2.get_move(g,len(g.all_cards[1]))
                elif(g.turn==3):m = player3.get_move(g,len(g.all_cards[2]))
                elif(g.turn==4):m = player4.get_move(g,len(g.all_cards[3]))
                r = g.play_card(m)
                #print(r)
            #if(player1.type=="human" or player2.type=="human"): print(m);g.print_game()
        if(g.score1>g.score2): win1 += 1
        elif(g.score1<g.score2): win2 += 1
        else: tie += 1
        print("score 1: ",g.score1)
        print("score 2: ",g.score2)
        del(g)
    print("Win 1 =",win1,"Win 2 =",win2,"Tie =",tie)
    return 0

## DQN Agent

In [5]:
class DQNAgent(object):

    def __init__(self,index):
        self.reward = 0
        self.gamma = 0.9
        self.dataframe = pd.DataFrame()
        self.short_memory = np.array([])
        self.agent_target = 1
        self.agent_predict = 0
        self.learning_rate = 0.001
        self.model = self.network()
        #self.model = self.network("weights.hdf5")
        self.epsilon = 0
        self.actual = []
        self.memory = []
        self.index = index
    def get_state(self, game):
        my_cards = np.zeros([4,13])
        i = 0
        for card in game.all_cards[game.turn-1]:
            for j in range(4):
                my_cards[j][i] = card
            i+=1
            
        played_cards = np.zeros([4,13])
        i=0
        for card in game.played_cards:
            played_cards[math.floor(i/13)][i%13] = card
            i+=1
        deck = np.zeros([4,13])
        
        for k in range(len(game.deck)):
            for i in range(13):
                deck[k][i] = game.deck[k]
        ato_type = np.dot(np.ones(52).reshape([4,13]),game.ATO)
        ato_count = np.dot(np.ones(52).reshape([4,13]),game.ATO_count)
        state = np.zeros([5,4,13])
        state[0]=my_cards
        state[1]=played_cards
        state[2]=deck
        state[3]=ato_type
        state[4]=ato_count
        #print(np.asarray(state))
        return np.asarray(state)
    def set_reward(self, old_delta, new_delta):
        self.reward = 0
        self.reward = (new_delta - old_delta) * 10
        if(self.index ==1 or self.index==3): self.reward *= -1 
        return self.reward

    def network(self, weights=None):
        model = get_model(cnn_filter_num=32,cnn_first_filter_size=5,l2_reg=1e-4,value_fc_size=13,cnn_filter_size=3,res_layer_num=5,n_labels=13)
        opt = Adam()
        losses = ['categorical_crossentropy']
        model.compile(optimizer=opt, loss=losses, loss_weights=[1.25])

        if weights:
            model.load_weights(weights)
        return model

    def remember(self, state, action, reward, next_state,done):
        self.memory.append((state, action, reward, next_state,done))

    def replay_new(self, memory):
        if len(memory) > 100:
            minibatch = random.sample(memory, 100)
        else:
            minibatch = memory
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state.reshape(1,5,4,13)))
            target_f = self.model.predict(state.reshape(1,5,4,13))
            target_f[0][np.argmax(action)] = target
            self.model.fit(state.reshape(1,5,4,13), target_f, epochs=1, verbose=0)
        

    def train_short_memory(self, state, action, reward, next_state, done):
        target = reward
        if not done:
            target = reward + self.gamma * np.amax(self.model.predict(next_state.reshape(1,5,4,13)))
        p = self.model.predict(state.reshape(1,5,4,13))
        #print(p.shape)
        target_f = self.model.predict(state.reshape(1,5,4,13))
        #print(target_f.shape,np.argmax(action))
        target_f[0][np.argmax(action)] = target
        self.model.fit(state.reshape(1,5,4,13), target_f, epochs=1, verbose=0)

## Neural Network

In [6]:
def get_model(cnn_filter_num,cnn_first_filter_size,l2_reg,value_fc_size,cnn_filter_size,res_layer_num,n_labels):
    in_x = x = Input((5,4,13))
    
    x = Conv2D(filters=cnn_filter_num, kernel_size=cnn_first_filter_size, padding= "same",
              data_format="channels_first", use_bias=False, kernel_regularizer=l2(l2_reg),
              name="input_conv"+str(cnn_first_filter_size)+"-"+str(cnn_filter_num))(x)
    x = BatchNormalization(axis=-1, name="input_batchnorm")(x)
    x = Activation("relu", name="input_relu")(x)
    
    for i in range(res_layer_num):
        x = build_residual_block(x, i+1,cnn_filter_num,l2_reg,cnn_filter_size)
    
    res_out = x
    
    x = Conv2D(filters=2, kernel_size=1, data_format="channels_first", use_bias=False, kernel_regularizer=l2(l2_reg),
              name="policy_conv-1-2")(res_out)
    x = BatchNormalization(axis=-1, name="policy_batchnorm")(x)
    x = Activation("relu", name="policy_relu")(x)
    x = Flatten(name="policy_flatten")(x)
    policy_out = Dense(n_labels,kernel_regularizer=l2(l2_reg), activation="softmax", name="policy_out")(x)
    """
    x = Conv2D(filters=4, kernel_size=1, data_format="channels_first", use_bias=False, kernel_regularizer=l2(l2_reg),
              name="value_conv-1-4")(res_out)
    x = BatchNormalization(axis=-1, name="value_batchnorm")(x)
    x = Activation("relu", name="value_relu")(x)
    x = Flatten(name="value_flatten")(x)
    x = Dense(value_fc_size, kernel_regularizer=l2(l2_reg), activation="relu", name="value_dense")(x)
    value_out = Dense(1, kernel_regularizer=l2(l2_reg), activation="tanh", name="value_out")(x)
    """
    return Model(in_x, policy_out, name="west_model")


def build_residual_block(x, index,cnn_filter_num,l2_reg,cnn_filter_size):
    in_x = x
    res_name = "res"+str(index)
    x = Conv2D(filters=cnn_filter_num, kernel_size=cnn_filter_size, padding="same",
              data_format="channels_first", use_bias = False, kernel_regularizer=l2(l2_reg),
              name=res_name+"_conv1-"+str(cnn_filter_size)+"-"+str(cnn_filter_num))(x)
    x = BatchNormalization(axis=-1, name=res_name+"_batchnorm1")(x)
    x = Activation("relu", name=res_name+"_relu1")(x)
    x = Conv2D(filters=cnn_filter_num, kernel_size=cnn_filter_size, padding="same",
              data_format="channels_first", use_bias = False, kernel_regularizer=l2(l2_reg),
              name=res_name+"_conv2-"+str(cnn_filter_size)+"-"+str(cnn_filter_num))(x)
    x = BatchNormalization(axis=-1, name=res_name+"_batchnorm2")(x)
    x = Add(name=res_name+"_add")([in_x,x])
    x = Activation("relu", name = res_name+"_relu2")(x)
    return x

## Trainer

In [7]:
def train(agents = [DQNAgent(0),DQNAgent(1),DQNAgent(2),DQNAgent(3)],trainer = Player("random"),numGames = 100):
    print("Training on",numGames," games.")
    score_plot = []
    counter_plot = []
    old_state = [np.zeros([5,4,52]),np.zeros([5,4,52]),np.zeros([5,4,52]),np.zeros([5,4,52])]
    for gm in range(numGames):
        print("Playing game #",gm+1," ...")
        g=Game(turn = random.randint(1,4))
        for i in range(4):
            agents[i].epsilon = numGames/2 - gm
        while(not g.finish):
            r = -1
            turn = g.turn
            ### save the vals of the table before the agent's turn
            if(turn == 1):
                old_state[g.turn-1] = agents[g.turn-1].get_state(g)
                old_delta = g.score1 - g.score2
            while(r==-1):
                if random.randint(0, int(numGames/2)) < agents[g.turn-1].epsilon or turn != 1:
                    m = random.randint(0,len(g.all_cards[g.turn-1])-1)
                    final_move = keras.utils.to_categorical(m, num_classes=13)
                    r = g.play_card(m)
                else:
                    old_state[g.turn-1].reshape(260)
                    m = np.argmax(agents[g.turn-1].model.predict(old_state[g.turn-1].reshape(1,5,4,13)))
                    final_move = keras.utils.to_categorical(m, num_classes=13)
                    #print("hereee!!!")
                    r = g.play_card(m)
                    while(r==-1):
                        m = random.randint(0,len(g.all_cards[g.turn-1])-1)
                        final_move = keras.utils.to_categorical(m, num_classes=13)
                        r = g.play_card(m)
            if(g.pack==1):
                ### save what happened and the generated reward
                ## each agent will get a part of memeory corresponding to what happend from their prespective
                new_state = agents[g.turn-1].get_state(g)
                new_delta = g.score1 - g.score2
                reward = agents[0].set_reward(old_delta,new_delta)
                agents[0].train_short_memory(old_state[0], final_move, reward, new_state, g.finish)
                agents[0].remember(old_state[0], final_move, reward, new_state,g.finish)  
                #for i in range(4):
                 #   reward = agents[i].set_reward(old_delta,new_delta)
                  #  agents[i].train_short_memory(old_state[i], final_move, reward, new_state, g.finish)
                   # agents[i].remember(old_state[i], final_move, reward, new_state,g.finish)        
        #for i in range(4):
        agents[0].replay_new(agents[0].memory)
        print("Game:",gm+1,"Scores:",g.score1,g.score2,"Aquired data:",len(agents[0].memory))#sum(len(agents[0].memory) ))#for i in range(4)))
        #score_plot.append(g.score2-g.score1)
        #counter_plot.append(gm)
        del(g)
        #input()
    print("Finished training!")
    #for i in range(4):
    #    save_frozen_graph(agents[i].model,"model"+str(i))
    #plot_seaborn(counter_plot, score_plot)
    return agents[0].model,agents[1].model,agents[2].model,agents[3].model

W0124 15:40:07.339300  6304 deprecation_wrapper.py:119] From c:\users\yazeed\appdata\local\programs\python\python35\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0124 15:40:07.357250  6304 deprecation_wrapper.py:119] From c:\users\yazeed\appdata\local\programs\python\python35\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0124 15:40:07.362238  6304 deprecation_wrapper.py:119] From c:\users\yazeed\appdata\local\programs\python\python35\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0124 15:40:07.385177  6304 deprecation_wrapper.py:119] From c:\users\yazeed\appdata\local\programs\python\python35\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is de

## Testing

In [8]:
#p2 = Player("DQN",agent=Agent,depth=20)
#p1 = Player("random",depth=5)
#p2 = Player("random",depth=5)
#p3 = Player("random",depth=5)
#p4 = Player("random",depth=5)
#run(p1,p2,p3,p4,100)
#Agent = DQNAgent()
#trainer = Player("minmax",depth=5)
m1,m2,m3,m4 = train(numGames=1)
#save_frozen_graph(m1,"west1.h5")

Training on 1  games.
Playing game # 1  ...


W0124 15:40:18.478291  6304 deprecation.py:323] From c:\users\yazeed\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Game: 1 Scores: 6 7 Aquired data: 13
Finished training!


## Saving model and Freezing graph

In [9]:
model = m1
import os
os.makedirs('./model', exist_ok=True)
model.save('./model/keras_model.h5')
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)
from keras.models import load_model
model = load_model('./model/keras_model.h5')
model.summary()

from keras import backend as K
import tensorflow as tf
sess = K.get_session()
graph_def = sess.graph.as_graph_def()
# graph_def
#show_graph(graph_def)
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph
from keras import backend as K
import tensorflow as tf

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, "model", "tf_model.pb", as_text=False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 5, 4, 13)     0                                            
__________________________________________________________________________________________________
input_conv5-32 (Conv2D)         (None, 32, 4, 13)    4000        input_1[0][0]                    
__________________________________________________________________________________________________
input_batchnorm (BatchNormaliza (None, 32, 4, 13)    52          input_conv5-32[0][0]             
__________________________________________________________________________________________________
input_relu (Activation)         (None, 32, 4, 13)    0           input_batchnorm[0][0]            
__________________________________________________________________________________________________
res1_conv1

W0124 15:40:31.953593  6304 deprecation.py:323] From <ipython-input-9-9dfcf6fdd690>:31: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0124 15:40:31.954613  6304 deprecation.py:323] From c:\users\yazeed\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\framework\graph_util_impl.py:270: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


'model\\tf_model.pb'